# pandapower Optimal Power Flow
This is an introduction into the usage of the pandapower optimal power flow. It shows how to set the constraints and the cost factors into the pandapower element tables.

## Example Network

We use the following four bus example network for this tutorial:

<img src="pics/example_opf.png" width="50%">

We first create this network in pandapower:

In [119]:
import pandapower as pp
import numpy as np
net = pp.create_empty_network()

#create buses
bus1 = pp.create_bus(net, vn_kv=220.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)

#create 220/110 kV transformer
pp.create_transformer(net, bus1, bus2, std_type="100 MVA 220/110 kV")
net.trafo.vkr_percent.at[0] = 0
net.trafo.pfe_kw.at[0]=0

#create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus4, bus2, length_km=40., std_type='149-AL1/24-ST1A 110.0')

#create loads
pp.create_load(net, bus2, p_mw=60, controllable=False)
pp.create_load(net, bus3, p_mw=70, controllable=False)
pp.create_load(net, bus4, p_mw=10, controllable=False)

#create generators
eg = pp.create_ext_grid(net, bus1, min_p_mw=-1000, max_p_mw=1000)
g0 = pp.create_gen(net, bus3, p_mw=80, min_p_mw=0, max_p_mw=80,  vm_pu=1.01, controllable=True)
g1 = pp.create_gen(net, bus4, p_mw=100, min_p_mw=0, max_p_mw=100, vm_pu=1.01, controllable=True)

In [120]:
net.trafo

,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,...,tap_neutral,tap_min,tap_max,tap_step_percent,tap_step_degree,tap_pos,tap_phase_shifter,parallel,df,in_service
0,None,100 MVA 220/110 kV,0,1,100.0,220.0,110.0,12.0,0.0,0.0,...,0,-9,9,1.5,0.0,0,False,1,1.0,True


## Loss Minimization

We specify the same costs for the power at the external grid and all generators to minimize the overall power feed in. This equals an overall loss minimization:

In [121]:
costeg = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=10)
costgen1 = pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
costgen2 = pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

We run an OPF:

In [122]:
pp.runopp(net, delta=1e-16)

This function runs an Optimal Power Flow using the PYPOWER OPF. To make sure that the PYPOWER OPF converges, we decrease the power tolerance `delta` (the default value is `delta=1e-10`). The power tolerance `delta` is a measure of the extent to which exceeding of minimum and maximum power limits is tolerated. That is, in above case, the limits considered by the OPF for the generators are `min_p_mw - delta` and `max_p_mw + delta` as lower and upper bound respectively on the active power. 

Let's check the results:

In [123]:
net.res_ext_grid

,p_mw,q_mvar
0,59.999633,2.55991


In [124]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,69.997330,-1.993949,-4.130836,0.999560
1,10.003033,-1.495470,-4.130649,0.999561


Since all costs were specified the same, the OPF minimizes overall power generation, which is equal to a loss minimization in the network. The loads at buses 3 and 4 are supplied by generators at the same bus, the load at Bus 2 is provided by a combination of the other generators so that the power transmission leads to minimal losses.

## Individual Generator Costs

Let's now assign individual costs to each generator.

We assign a cost of 10 ct/kW for the external grid, 15 ct/kw for the generator g0 and 12 ct/kw for generator g1:

In [125]:
net.poly_cost.cp1_eur_per_mw.at[costeg] = 10
net.poly_cost.cp1_eur_per_mw.at[costgen1] = 15
net.poly_cost.cp1_eur_per_mw.at[costgen2] = 12

And now run an OPF:

In [126]:
pp.runopp(net, delta=1e-16)

We can see that all active power is provided by the external grid: 

In [127]:
net.res_ext_grid

,p_mw,q_mvar
0,143.925799,9.608184


In [128]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000075,8.346090,-16.315952,0.970337
1,0.000205,10.176077,-13.387207,0.992411


This makes sense, because the external grid has the lowest cost of all generators and we did not define any constraints.

The dispatch costs are given in net.res_cost:

In [129]:
net.res_cost

1439.26157143092

### Transformer Constraint

Since all active power comes from the external grid and subsequently flows through the transformer, the transformer is overloaded with a loading of about 145%:

In [130]:
net.res_trafo.loading_percent

0    144.247362
Name: loading_percent, dtype: float64

# Adding all the equality constraints into the optimum power flow.

In [131]:
def constraints(loading_trafo,loading_line,min_bus_voltage_pu,max_bus_volatge_pu ):

    net.trafo["max_loading_percent"]=loading_trafo
    net.line["max_loading_percent"]=loading_line
    net.bus["min_vm_pu"] = min_bus_voltage_pu
    net.bus["max_vm_pu"] = max_bus_volatge_pu
    
    


In [172]:
constraints(loading_trafo=70,loading_line=70,min_bus_voltage_pu=0.98,max_bus_volatge_pu=1.02)
net.trafo


,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,...,tap_min,tap_max,tap_step_percent,tap_step_degree,tap_pos,tap_phase_shifter,parallel,df,in_service,max_loading_percent
0,None,100 MVA 220/110 kV,0,1,100.0,220.0,110.0,12.0,0.0,0.0,...,-9,9,1.5,0.0,0,False,1,1.0,True,70


We now limit the transformer loading to 70%:

In [173]:
#net.trafo["max_loading_percent"] = 50

(the max_loading_percent parameter can also be specified directly when creating the transformer)
and run the OPF:

In [174]:
pp.runopp(net, delta=1e-16)

We can see that the transformer complies with the maximum loading:

In [175]:
net.res_trafo.loading_percent

0    70.000026
Name: loading_percent, dtype: float64

And power generation is now split between the external grid and generator 1 (which is the second cheapest generation unit):

In [176]:
net.res_ext_grid

,p_mw,q_mvar
0,69.989228,-1.016535


In [177]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,2.412234,3.405295,-8.225986,0.983555
1,69.999973,3.295989,-3.844231,1.018560


This comes of course with an increase in dispatch costs:

In [178]:
net.res_cost

1576.0755790150336

### Line Loading Constraints

Wen now look at the line loadings:

In [179]:
net.res_line.loading_percent

0    27.014832
1    50.124491
2    16.369946
Name: loading_percent, dtype: float64

and run the OPF with a 50% loading constraint:

In [180]:
#net.line["max_loading_percent"] = 50
pp.runopp(net, delta=1e-16)

Now the line loading constraint is complied with:

In [181]:
net.res_line.loading_percent

0    27.014832
1    50.124491
2    16.369946
Name: loading_percent, dtype: float64

And all generators are involved in supplying the loads:

In [182]:
net.res_ext_grid

,p_mw,q_mvar
0,69.989228,-1.016535


In [183]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,2.412234,3.405295,-8.225986,0.983555
1,69.999973,3.295989,-3.844231,1.018560


This of course comes with a once again rising dispatch cost:

In [184]:
net.res_cost

1576.0755790150336

### Voltage Constraints

Finally, we have a look at the bus voltage:

In [185]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.000000,0.000000,-69.989228,1.016535,10.000000,-2.199488e-22
1,1.004772,-4.794930,60.000000,0.000000,14.180192,1.892066e-02
2,0.983555,-8.225986,67.587766,-3.405295,14.999969,4.469826e-22
3,1.018560,-3.844231,-59.999973,-3.295989,13.950166,6.505646e-22


and constrain it:

In [186]:
net.bus["min_vm_pu"] = 1.0
net.bus["max_vm_pu"] = 1.02
pp.runopp(net, delta=1e-16)

We can see that all voltages are within the voltage band:

In [187]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.000000,0.000000,-69.861067,4.307786,10.000000,-2.934400e-22
1,1.008695,-4.767472,60.000000,0.000000,14.096900,-3.866703e-02
2,1.000000,-8.444922,67.391468,-17.315136,14.999987,1.169963e-21
3,1.020000,-3.752441,-59.999988,7.262510,13.880001,-4.869861e-22


And all generators are once again involved in supplying the loads:

In [188]:
net.res_ext_grid

,p_mw,q_mvar
0,69.861067,-4.307786


In [189]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,2.608532,17.315136,-8.444922,1.00
1,69.999988,-7.262510,-3.752441,1.02


This of course comes once again with rising dispatch costs:

In [190]:
net.res_cost

1577.7385414415235

## DC OPF

pandapower also provides the possibility of running a DC Optimal Power Flow:

In [191]:
pp.rundcopp(net, delta=1e-16)

Since voltage magnitudes are not included in the DC power flow formulation, voltage constraints cannot be considered in the DC OPF:

In [192]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.000000,0.000000,-69.999996,4.307786,10.000000,0.0
1,1.008695,-4.812932,60.000000,0.000000,14.045738,0.0
2,1.000000,-8.125492,69.999991,-17.315136,14.045739,0.0
3,1.020000,-3.696611,-59.999994,7.262510,14.045738,0.0


Line and transformer loading limits are however complied with:

In [193]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,24.374996,0.0,-24.374996,0.0,0.0,0.0,0.127936,0.127936,0.127936,1.0,-4.812932,1.0,-8.125492,27.220330
1,-45.624994,0.0,45.624994,0.0,0.0,0.0,0.239469,0.239469,0.239469,1.0,-8.125492,1.0,-3.696611,50.950876
2,14.375000,0.0,-14.375000,0.0,0.0,0.0,0.075449,0.075449,0.075449,1.0,-3.696611,1.0,-4.812932,16.053017


In [194]:
net.res_trafo

,p_hv_mw,q_hv_mvar,p_lv_mw,q_lv_mvar,pl_mw,ql_mvar,i_hv_ka,i_lv_ka,vm_hv_pu,va_hv_degree,vm_lv_pu,va_lv_degree,loading_percent
0,69.999996,0.0,-69.999996,0.0,0.0,0.0,0.183702,0.367405,1.0,0.0,1.0,-4.812932,69.999996


As are generator limits:

In [195]:
net.gen

,name,bus,p_mw,vm_pu,sn_mva,min_q_mvar,max_q_mvar,scaling,slack,in_service,slack_weight,type,controllable,min_p_mw,max_p_mw,power_station_trafo
0,None,2,80.0,1.01,NaN,NaN,NaN,1.0,False,True,0.0,None,True,0.0,80.0,NaN
1,None,3,100.0,1.01,NaN,NaN,NaN,1.0,False,True,0.0,None,True,0.0,100.0,NaN


In [196]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000009,17.315136,-8.125492,1.0
1,69.999994,-7.262510,-3.696611,1.0


The cost function is the same for the linearized OPF as for the non-linear one:

In [197]:
net.res_cost

1540.000051228771

## Piecewise linear cost functions

The OPF also offers piecewise linear cost functions. Let us first check the actual cost function setup:

In [198]:
net.poly_cost

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2


An element can either have polynomial costs or piecewise linear costs at the same time. So let us first delete the polynomial costs in order to avoid confusion and errors:

In [199]:
net.poly_cost.drop(net.poly_cost.index.values, inplace=True)

The results above have been produced with linear polynomial cost functions. Let's try to reproduce the results using piecewise linear cost functions. Costs have to be defined for the whole range of generators and external grids:

In [200]:
net.gen[["min_p_mw", "max_p_mw"]]

,min_p_mw,max_p_mw
0,0.0,80.0
1,0.0,100.0


In [201]:
net.ext_grid[["min_p_mw", "max_p_mw"]]

,min_p_mw,max_p_mw
0,-1000.0,1000.0


We define the piecewise linear cost as constant over the whole range to reproduce the polyomial costs defined above:

In [202]:
pp.create_pwl_cost(net, 0, "gen", [[0, 80, 15]])
pp.create_pwl_cost(net, 1, "gen", [[0, 100, 12]])
pp.create_pwl_cost(net, 0, "ext_grid", [[-1000, 1000, 10]])

UserWarning: There already exist costs for gen 0

Let us check the results from the previous OPF again!

In [ ]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,2.929334e-07,20.806717,-6.070869,1.0
1,9.000000e+01,-13.901088,-1.156632,1.0


In [ ]:
net.res_cost

1580.0000017576074

We run the same OPF now with different cost function setup. We should get the exact same results:

In [ ]:
pp.rundcopp(net, delta=1e-16)

In [ ]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,2.929334e-07,20.806717,-6.070869,1.0
1,9.000000e+01,-13.901088,-1.156632,1.0


In [ ]:
net.res_cost

1580.0000017576074

Now lets define real piecewise linear costs for generator 1. We define the costs as 12€/MW up to 70MW and 20€/MW from 70MW to 100MW:

In [ ]:
net.pwl_cost.points.loc[1] = [[0, 70, 12], [70, 100, 20]]

And run another OPF:

In [ ]:
pp.rundcopp(net, delta=1e-16)

Now we can see that generator 1 only dispatches 70MW, above which generator 0 becomes less expensive and is therefore dispatched:

In [ ]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,20.000006,20.806717,-5.221494,1.0
1,69.999997,-13.901088,-1.641988,1.0


## Debugging

For more information on the status of the OPF solver, set verbose=True:

In [ ]:
pp.runopp(net, verbose=True, delta=1e-16)

PYPOWER Version 5.1.4, 27-June-2018 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!

Converged in 0.75 seconds
Objective Function Value = 1664.15 $/hr
| PyPower (ppci) System Summary - these are not valid for pandapower DataFrames|

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              4     Total Gen Capacity    1180.0       -3000000000.0 to 3000000000.0
Generators         3     On-line Capacity      1180.0       -3000000000.0 to 3000000000.0
Committed Gens     3     Generation (actual)    141.6               1.0
Loads              3     Load                   140.0               0.0
  Fixed            3       Fixed                140.0               0.0
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches           4   

In [203]:
from sympy import symbols, Eq, solve